# Image classifiers training
## 1. Data Augmentation

In [1]:
import numpy as np
import torch
import torchvision
import torchvision.transforms as transforms

train_dir = './train_images'
test_dir = './test_images'

# Define the transformations for augmentation
data_transforms = {
    'train': transforms.Compose([
        transforms.Grayscale(), 
        transforms.RandomResizedCrop(36),
        transforms.RandomHorizontalFlip(),
        transforms.ToTensor(),
        transforms.Normalize(mean=(0,),std=(1,))
    ]),
    'test':transforms.Compose([
        transforms.Grayscale(),
        transforms.ToTensor(),
        transforms.Normalize(mean=(0,),std=(1,))
    ])        
}
train_data = torchvision.datasets.ImageFolder(train_dir, transform=data_transforms['train'])
test_data = torchvision.datasets.ImageFolder(test_dir, transform=data_transforms['test'])

train_data_size = len(train_data)
test_data_size = len(test_data)

In [2]:
# Set device
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

In [3]:
from torch.utils.data.sampler import SubsetRandomSampler

valid_size = 0.2
batch_size = 32
seed = 42
np.random.seed(seed)
torch.manual_seed(seed)

num_train = len(train_data)
indices_train = list(range(num_train))
np.random.shuffle(indices_train)
split_tv = int(np.floor(valid_size * num_train))
train_new_idx, valid_idx = indices_train[split_tv:],indices_train[:split_tv]

train_sampler = SubsetRandomSampler(train_new_idx)
valid_sampler = SubsetRandomSampler(valid_idx)

train_loader = torch.utils.data.DataLoader(train_data, batch_size=batch_size, sampler=train_sampler, num_workers=5)
valid_loader = torch.utils.data.DataLoader(train_data, batch_size=batch_size, sampler=valid_sampler, num_workers=5)
test_loader = torch.utils.data.DataLoader(test_data, batch_size=batch_size, shuffle=True, num_workers=5)
classes = ('noface','face')

In [4]:
import torch.nn as nn
import torch.nn.functional as F


class Net_4_layers(nn.Module):
    def __init__(self):
        super(Net_4_layers, self).__init__()
        self.conv1 = nn.Conv2d(1, 6, 5)
        self.pool = nn.MaxPool2d(2, 2)
        self.conv2 = nn.Conv2d(6, 16, 5)
        self.fc1 = nn.Linear(16 * 6 * 6, 64)
        self.fc2 = nn.Linear(64, 32)
        self.fc3 = nn.Linear(32, 16) # Add a new fully connected layer
        self.fc4 = nn.Linear(16, 2)

    def forward(self, x):
        x = self.pool(F.relu(self.conv1(x)))
        x = self.pool(F.relu(self.conv2(x)))
        x = x.view(-1, 16 * 6 * 6)
        x = F.relu(self.fc1(x))
        x = F.relu(self.fc2(x))
        x = F.relu(self.fc3(x))
        x = self.fc4(x)
        return x

model_1 = Net_4_layers().to(device)

In [5]:
def train_NN(n_epochs, optimizer, model, criterion, early_stop_value, scheduler):
    EPOCHS_WAIT_BEFORE_EARLY_STOPPING = 5 # tolerate some epoch with no improvement to avoid local minimum
    epochs_count = 0

    for epoch in range(1, n_epochs+1):  # loop over the dataset multiple times
        running_loss = 0.0
        for i, data in enumerate(train_loader, 0):
            # get the inputs; data is a list of [inputs, labels]
            inputs, labels = data
            inputs = inputs.to(device)
            labels = labels.to(device)
            # zero the parameter gradients
            optimizer.zero_grad()

            # forward + backward + optimize
            outputs = model(inputs)
            loss = criterion(outputs, labels)
            loss.backward()
            optimizer.step()

            running_loss += loss.item()
        
        loss_average = running_loss / len(train_loader)
        print(f'Epoch [{epoch}/{n_epochs}], Batch [{i + 1}/{len(train_loader)}], Loss: {loss_average:.3f}, Learning Rate: {scheduler.get_last_lr()[0]:.6f}')
        
        # stop the training if the loss is lower than the threshold
        if (loss_average < early_stop_value):
            epochs_count = epochs_count + 1
            if (epochs_count == EPOCHS_WAIT_BEFORE_EARLY_STOPPING):
                break

        scheduler.step()

    print('Finished Training')

In [ ]:
import torch.optim as optim
from torch.optim.lr_scheduler import StepLR

criterion = nn.CrossEntropyLoss()
optimizer_Adam = optim.Adam(model_1.parameters(), lr=0.001, weight_decay=1e-4)
scheduler = StepLR(optimizer_Adam, step_size=5, gamma=0.8)

n_epochs = 30
early_stop_value = 0.1

train_NN(n_epochs, optimizer_Adam, model_1, criterion, early_stop_value, scheduler)

# save the model after training
# path_1 = "models/4-layer-lr-scheduling-net.pth"
# torch.save(model_1.state_dict(), path_1)

In [5]:
def get_classifier_accuracy(model):
    # number of no face and face images in the test dataset
    NO_FACE = 6831
    FACE = 797

    correct = 0
    correct_no_face = 0
    correct_has_face = 1
    total = 0
    with torch.no_grad():
        for data in test_loader:
            inputs, labels = data
            inputs = inputs.to(device)
            labels = labels.to(device)
            outputs = model(inputs)
            _, predicted = torch.max(outputs.data, 1)
            total += labels.size(0)
            correct_no_face += ((predicted == 0) & (labels == 0)).sum().item()
            correct_has_face += ((predicted == 1) & (labels == 1)).sum().item()
            correct += (predicted == labels).sum().item()
    
    print("correct no face = {:.2f}, has face = {:.2f}".format(100 * correct_no_face / NO_FACE, 100 * correct_has_face / FACE))

    return 100 * correct / total

In [6]:
model_1.load_state_dict(torch.load('models/4-layer-lr-scheduling-net.pth'))

model_1_accuracy = get_classifier_accuracy(model_1)
print('Accuracy of the 4-layer classifier on the {} test images: {:.2f}'.format(test_data_size, model_1_accuracy))

correct no face = 99.00, has face = 89.84
Accuracy of the 4-layer classifier on the 7628 test images: 98.03
